In [ ]:
import numpy as np

In [ ]:
import zipfile
from shutil import copyfile
import glob
import re
import h5py
import green2
import json
import os
import fnmatch

In [ ]:
from BachataClass_upd import *
from wavesensors import *
from wavesource import *

In [ ]:
from rndflow import job

In [ ]:
fmin, fmax = 10, 30
xc = 0

In [ ]:
# sorting function key
def limits(key_string):
    labels = [['a','low','lower','min','left','down','l','1'],
              ['b','high', 'up','upper','right','max','r','2']]
    
    for i in [0,1]:
        if (key_string.lower()[1:] in labels[i]) or (key_string.lower()[2:] in labels[i]):
            return key_string.lower()[0]+labels[i][0]

In [ ]:
#--------------------------------------------------------------------------
#  Read zip file functions
#--------------------------------------------------------------------------
def read_hit_pts(filename):
    
    with open(filename, 'r') as f:
        s = f.readlines()
    f.close()

    res = np.zeros((len(s), 6))
    geo_coords=list()
    names=[]
    
    for i in range(len(s)):
        line=s[i].split()
        res[i, :] = np.array([int(i) for i in line[1:7]])
        geo_coords.append([float(line[8]),float(line[7]),float(line[6])]) # add real X, real Y, depth, changed axis between XY
        names.append(line[0])

    xy_cells = (res[:, 0:2])
    xy_metres = (res[:, 2:4])
    zz = (res[:, 4:])

    return xy_cells, xy_metres, zz, geo_coords, names

In [ ]:
#------------------------------------------------------------------------#
#  _____________________________BEGIN__________________________________  #
#------------------------------------------------------------------------#
# Load everything from input package:
globals().update(job.load())    #load variables from input package
globals().update(job.params())  #load parameters from input package
pkg=job.packages()[0]
fields=pkg.fields

In [ ]:
files = glob.glob('**',recursive = True)

In [ ]:
zip_file = [name for name in files if '.zip' in name]

In [ ]:
if len(zip_file)>0:
    try:
        with zipfile.ZipFile(*zip_file) as z:
            z.extractall(os.path.dirname(*zip_file))
            print("Extracted all")
    except:
        print("Invalid file")

In [ ]:
cellsize=fields['cell_size']

In [ ]:
#---------------------------------------
#read source points coordinates
files = glob.glob('**',recursive = True)
source_path=[s for s in files if fnmatch.fnmatch(s, '*.txt')]
source_file=[s for s in source_path if 'points' in s][0]
print('Reading source points coords')
sourcexy_c, sourcexy_m, zz, geo_coords_source, source_names = read_hit_pts(source_file)

In [ ]:
x_center,y_center=np.mean(np.array(geo_coords_source)[:,:2],axis=0)
print(x_center,y_center)

In [ ]:
#test
if np.sum(sourcexy_c*cellsize - sourcexy_m, axis=(0,1)) == 0:
    print('Data is correct!')
else:
    raise ValueError('Source points data is not coherent! Abort...')

In [ ]:
xy_source = np.append(sourcexy_m, zz[:,1][:,None], axis=1)
del sourcexy_m
print(xy_source)

In [ ]:
#---------------------------------------
#read sensors.txt
sensors_path=[s for s in files if 'sensors.txt' in s][0]
print('Reading sensors coords')
sensorxy_c, sensorxy_m, zz, geo_coords_sens,sens_names = read_hit_pts(sensors_path)

In [ ]:
print(geo_coords_sens[:3])

In [ ]:
#test
if np.sum(sensorxy_c*cellsize - sensorxy_m, axis=(0,1)) == 0:
    print('Data is correct!')
else:
    raise ValueError('Sensors data is not coherent! Abort...')

In [ ]:
xy_sensors = np.append(sensorxy_m, zz[:,1][:,None], axis=1)
del sensorxy_m
print(xy_sensors[:3])

In [ ]:
field_lst=list(map(lambda name,coord: [int(name),*coord],sens_names,geo_coords_sens))
print(field_lst[:3])
# ----------------Initialization of source object -------------------------
source = wavesource(xx=xy_source)

In [ ]:
# ----------------Initialization of sensors array object ------------------
sensors = wavesensors(xx=xy_sensors)

In [ ]:
#----------------Calculation domainX, domainY, domainZ, --------------------
shape=np.unique(xy_source[:,0]).shape[0]
X=np.sort(np.unique(xy_source[:,0]))
Y=np.sort(np.unique(xy_source[:,1]))
Z=np.sort(np.unique(xy_source[:,2]))
width_x=X[shape//2]-X[0]
width_y=Y[shape//2]-Y[0]
domainX=np.linspace((xc-width_x),(xc+width_x),shape)
domainY=np.linspace((xc-width_y),(xc+width_y),shape)
domainZ=Z
#----------------Write data for batchata -----------------------------------
vp=7200
vs=vp/2
fd=500
T=5
Q=21
Nd = 2

sens_geo_coords=np.array(field_lst)[:,1:]
xx=sens_geo_coords-geo_coords_source

# -----------BEFORE CORRECT
rs = green2.response_t(xx, T=T, fs=fd)
rs.make_green(Q=Q, vp=vp, vs=vs, fp=1e7, a3=1e-5, naive=False)

Gtp = rs.Gtp(nD=Nd)
Gts = rs.Gts(nD=Nd)
Gtsum = -Gtp + Gts

data_final=[]
for i in range(sensors.coords.shape[0]):
    data_final.extend(Gtsum[i,[2,0,1],:,:].reshape(3,1,6,rs.N))
# ----------------Initialization of bachata-object -------------------------
filename = f'batch_Q{Q}_VP{vp}_VS{vs}_Nd{Nd}_T{T}_fd{fd}_new.hdf5'

kwargs = {'data_type': 'full_wave_time',
            'channels': ['Z','X','Y'],
            'L': rs.N,
            'fd': fd,
            'xcenter': x_center,
            'ycenter': y_center,
            'grid_step': np.array([cellsize]*3,dtype=int),
            'altitude': np.array([source.origin[2]],dtype=float),
            'mult_degree': np.array([10**power]),
            'sensor': sens_names, 
            'domainX': domainX,
            'domainY': domainY,
            'domainZ': domainZ,
            'field': np.array(field_lst)[:,:3],
            'fmin': rs.ff.min(),
            'fmax': rs.ff.max(),
            'data': data_final 
            }                   # add power 10000 units

print(f'VP {vp}, VS {vs}')
jpath=job.save_package(label=pkg.label)
fpath = str(jpath)+'/files/'
os.mkdir(fpath)

obj = BachataClass(fpath+filename,**kwargs)
print('Done')

In [ ]:
def integral(signal):
    
    integ=[]
    for i in range(1,signal.shape[0]):
        r=(signal[i]+signal[i-1])*fd
        integ.append(r)
    res=np.array(integ)
    return res

In [ ]:
full_mod=h5py.File('batch_obj_0.001.hdf5')
full_mod['Channels'][sensor]['data'][0,tensor_comp,:].shape

In [ ]:
full_mod.keys()

In [ ]:
full_mod['L'][:]

In [ ]:
full_mod['Channels'][sensor]['data'][0,tensor_comp,:].shape

In [ ]:
import matplotlib.pyplot as plt
import h5py
from scipy import signal

sensor='1030_Z'
tensor_comp=2
samples=1500

files = glob.glob('**',recursive = True)
path_to_full_modelator=[f for f in files if filename in f][0]
print(path_to_full_modelator)

simple_mod=h5py.File(path_to_full_modelator)
full_mod=h5py.File('batch_obj_0.0005.hdf5') # power=1e4, modelling_step=0.0005 sig_len=5000
#full_mod=h5py.File('batch_obj_0.001.hdf5')  # power=2e4, modelling_step=0.001 sig_len=5000

newton_power=1e4
mult_degree=1e11
fd=500

k=newton_power*mult_degree#*(1/fd)

# simple plot
#plt.plot(simple_mod['Channels'][sensor]['data'][0,tensor_comp,:samples],label='simple modelator')
#plt.plot(full_mod['Channels'][sensor]['data'][0,tensor_comp,:samples],label='full wave modelator')

# centre explosion
sum_simple_mod=np.sum(simple_mod['Channels'][sensor]['data'][0,:3,:samples],axis=0)
sum_full_mod=np.sum(full_mod['Channels'][sensor]['data'][0,:3,:samples],axis=0)
#plt.plot(sum_simple_mod,label='sum, simple modelator')
#plt.plot(sum_full_mod,label='sum, full wave modelator')

# filtration
fs=10
sos = signal.butter(10, fs, 'lowpass', fs=fd, output='sos')

plt.plot(signal.sosfilt(sos, sum_simple_mod)*k,label='sum, simple modelator')
plt.plot(integral(signal.sosfilt(sos, sum_full_mod)),label="integral(sum, full modelator)")

plt.xlabel('samples')
plt.grid()
plt.legend()

In [ ]:
full_mod

In [ ]:
samples=2500

full_mod=h5py.File('batch_obj.hdf5') # power=1e4, modelling_step=0.001 sig_len=5000
full_mod_1=h5py.File('batch_obj_0.0005.hdf5') # power=1e4, modelling_step=0.0005 sig_len=5000
full_mod_2=h5py.File('batch_obj_0.001.hdf5')  # power=2e4, modelling_step=0.001 sig_len=5000

full_mod=full_mod['Channels'][sensor]['data'][0,2,:samples]
full_mod_1=full_mod_1['Channels'][sensor]['data'][0,2,:samples]
full_mod_2=full_mod_2['Channels'][sensor]['data'][0,2,:samples]

# sum_full_mod=np.sum(full_mod['Channels'][sensor]['data'][0,:3,:samples],axis=0)
# sum_full_mod_1=np.sum(full_mod_1['Channels'][sensor]['data'][0,:3,:samples],axis=0)
# sum_full_mod_2=np.sum(full_mod_2['Channels'][sensor]['data'][0,:3,:samples],axis=0)

#plt.plot(integral(signal.sosfilt(sos, sum_full_mod)),label="integral(sum, full modelator pow 1e4, m_s 0.001)")
#plt.plot(integral(signal.sosfilt(sos, sum_full_mod_1))+1,label="integral(sum, full modelator pow 1e4, m_s 0.0005)")
#plt.plot((integral(signal.sosfilt(sos, sum_full_mod_2))-1)/2,label="integral(sum, full modelator pow 2e4, m_s 0.001)")

plt.plot(full_mod,label="full modelator pow 1e4, m_s 0.001")
plt.plot(full_mod_1+1,label="full modelator pow 1e4, m_s 0.0005")
plt.plot(full_mod_2-1,label="full modelator pow 2e4, m_s 0.001")


plt.xlabel('samples')
plt.grid()
plt.legend()